# Kemal Demirel
# 191104091

In [8]:
import cv2
from PIL import Image
import os
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
def preprocess_image(image):
    image = cv2.resize(image, (300,300) )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # For RGB
    return image

In [3]:
def traverse_folder_to_preprocess(path):
    images = []
    for file in os.listdir(path):
        image = cv2.imread(os.path.join(path, file))
        image = preprocess_image(image)
        images.append(image)
    return images

In [4]:
# Pokemon resimlerinin bulunduğu path
pokemon_path = os.getcwd()+"/pokemon/"
pokemon_preprocessed_images = traverse_folder_to_preprocess(pokemon_path)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

In [17]:
# QI resimlerinin bulunduğu path
qi_path = os.getcwd()+"/QI/"
qi_preprocessed_images = traverse_folder_to_preprocess(qi_path)

In [5]:
sift = cv2.xfeatures2d.SIFT_create()

[ WARN:0@6.096] global shadow_sift.hpp:15 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [6]:
dataset_features = []

for image in pokemon_preprocessed_images:
    kp, desc = sift.detectAndCompute(image, None)
    dataset_features.append(desc)
    
dataset_features = np.vstack(dataset_features)

In [7]:
num_clusters = 100
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(dataset_features) # random state silmeyi dene olmazsa

In [11]:
print(len(dataset_features))

249251


In [12]:
dataset_histograms = []
i=0
for desc in dataset_features:
    labels = kmeans.predict(desc.reshape(1, -1))
    hist, _ = np.histogram(labels, bins=num_clusters, range=(0, num_clusters-1))
    dataset_histograms.append(hist)
    i = i + 1
    if i%10 == 0:
        print(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [18]:
query_histograms = []
for image in qi_preprocessed_images:
    kp, desc = sift.detectAndCompute(image, None)
    labels = kmeans.predict(desc)
    hist, _ = np.histogram(labels, bins=num_clusters, range=(0, num_clusters-1))
    hist = hist.astype(float) / np.sum(hist)
    query_histograms.append(hist)
#query_histograms = np.vstack(query_histograms)

In [21]:
similarities = []
for query_hist in query_histograms:
    similarity = cosine_similarity(query_hist.reshape(1, -1), dataset_histograms)
    similarities.append(similarity[0])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.04714045 0.04714045 0.         ... 0.         0.0942809  0.0942809 ]


In [20]:
# Rank the dataset images by their similarity to each query image and display the top 3
for i in range(len(qi_preprocessed_images)):
    query_img = qi_preprocessed_images[i]
    similarity_scores = similarities[i]
    ranked_indices = np.argsort(similarity_scores)[::-1][:3]
    ranked_images = [pokemon_preprocessed_images[idx] for idx in ranked_indices]
    
    # Display the query image and the 3 most similar images
    cv2.imshow("Query Image", query_img)
    for j in range(len(ranked_images)):
        ranked_img = ranked_images[j]
        cv2.imshow("Similar Image {}".format(j+1), ranked_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

IndexError: list index out of range